In [ ]:
import os
import pathlib

import sqlalchemy as sa
from dotenv import dotenv_values, load_dotenv

from sqlalchemy.orm import DeclarativeBase, Session

In [ ]:
curPath = os.getcwd()
parPath = pathlib.Path(curPath).parent
dotEnvPath = os.path.join(parPath, ".env.dev")

print(dotEnvPath)
if not os.path.isfile(dotEnvPath):
    raise FileNotFoundError(f"File not found: {dotEnvPath}")

load_dotenv(dotenv_path=pathlib.Path(dotEnvPath), override=True)

In [ ]:
db_username = os.getenv("PSQL_USERNAME")
db_password = os.getenv("PSQL_PASSWORD")
db_database = os.getenv("PSQL_DATABASE")
db_host = os.getenv("PSQL_HOST")
db_port = os.getenv("PSQL_PORT")

print(db_username, db_password, db_database, db_host, db_port)

In [ ]:
urlString = (
    f"postgresql+pg8000://{db_username}:{db_password}@{db_host}:{db_port}/{db_database}"
)
engine = sa.create_engine(urlString)

In [ ]:
select_query = """
    SELECT DISTINCT ON (hospital_id) path, hospital_id, id
FROM fp_models 
WHERE is_active = true
    """

with engine.connect() as db_conn:
    # 1. Find all records with the same hospital_id and is_active = true

    active_rows = db_conn.execute(sa.text(select_query)).fetchall()

print(active_rows)

In [ ]:
class Base(DeclarativeBase):
    pass

class FpModel(Base):
    __tablename__ = "fp_models"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String)
    path = sa.Column(sa.String)

In [ ]:
session = Session(engine)

In [ ]:
stmt = sa.select(FpModel)
for fp in session.scalars(stmt):
    print(fp.id, fp.name, fp.path)